# Learn to plot Bayesian Network

In [1]:
!pip install cython
!pip install "vose<0.0.2" --no-build-isolation
!pip install sorobn
!pip install graphviz


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached vose-0.0.1.tar.gz (129 kB)
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'error'


  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [91 lines of output]
      Compiling vose/sampler.pyx because it changed.
      [1/1] Cythonizing vose/sampler.pyx
      
      Error compiling Cython file:
      ------------------------------------------------------------
      ...
      cimport numpy as np
      
      cdef class Sampler:
      
          cdef int n
          cdef np.int_t [:] alias
               ^
      ------------------------------------------------------------
      
      vose\sampler.pxd:6:9: 'int_t' is not a type identifier
      
      Error compiling Cython file:
      ------------------------------------------------------------
      ...
          cdef int n
          cdef np.int_t [:] alias
          cdef np.float_t [:] proba
      
          cdef int sample_1(self)
          cdef np.int_t [:] sample_k(self, int k)
               ^
      ---------------------------------------

  Using cached sorobn-0.3.0-py3-none-any.whl.metadata (551 bytes)
  Using cached numpy-1.26.4-cp310-cp310-win_amd64.whl.metadata (61 kB)
  Using cached pandas-1.5.3-cp310-cp310-win_amd64.whl.metadata (12 kB)
  Using cached vose-0.0.1.tar.gz (129 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [93 lines of output]
      Compiling vose/sampler.pyx because it changed.
      [1/1] Cythonizing vose/sampler.pyx
      
      Error compiling Cython file:
      ------------------------------------------------------------
      ...
      cimport numpy as np
      
      cdef class Sampler:
      
          cdef int n
          cdef np.int_t [:] alias
               ^
      ------------------------------------------------------------
      
      vose\sampler.pxd:6:9: 'int_t' is not a type identifier
      
      Error compiling Cython file:
      ------------------------------------------------------------
      ...
          cdef int n
          cdef np.int_t [:] alias
          cdef np.float_t [:] proba
      
          cdef int sample_1(self)
          cdef np.int_t [:] sample_k(self, int k)
               ^
      ---------------------------------------


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import graphviz

dot = graphviz.Digraph(comment='alarm')
dot.node('A', 'Burglary')
dot.node('B', 'Earthquake')
dot.node('C', 'Alarm')
dot.node('D', 'John calls')
dot.node('E', 'Mary calls')

dot.edges(['AC', 'BC', 'CD', 'CE'])

In [ ]:
print(dot.source)

In [ ]:
dot.render('round_table', directory='figures', format='svg', cleanup=True, view=True)
dot

**Task 1: convert the graph to Laplacian matrix (in degree), do this either on paper or just write out the matrix here**

# Bayesian Network consutrction and inference

Read "alarm.png" file from the github link, use sorobn to construct the bayesian network

In [ ]:
import sorobn as hh
import pandas as pd

bn = hh.BayesNet(
    ('Burglary', 'Alarm'),
    ('Earthquake', 'Alarm'),
    ('Alarm', 'John calls'),
    ('Alarm', 'Mary calls'),
    seed=42
  )

# P(Burglary)
bn.P['Burglary'] = pd.Series({False: .999, True: .001}) # P(Burglary=True) = 0.001, P(Burglary=False) = 0.999

# P(Earthquake)
bn.P['Earthquake'] = pd.Series({False: .998, True: .002}) # P(Earthquake=True) = 0.002, P(Earthquake=False) = 0.998

# P(Alarm | Burglary, Earthquake)
bn.P['Alarm'] = pd.Series({
              (True, True, True): .95, # P(Alarm=True | Burglary=True, Earthquake=True) = 0.95
              (True, True, False): .05, # P(Alarm=False | Burglary=True, Earthquake=True) = 0.05

              (True, False, True): .94, # P(Alarm=True | Burglary=True, Earthquake=False) = 0.94
              (True, False, False): .06, # P(Alarm=False | Burglary=True, Earthquake=False) = 0.06

              (False, True, True): .29, # P(Alarm=True | Burglary=False, Earthquake=True) = 0.29
              (False, True, False): .71, # P(Alarm=False | Burglary=False, Earthquake=True) = 0.71

              (False, False, True): .001, # P(Alarm=True | Burglary=False, Earthquake=False) = 0.001 
              (False, False, False): .999 # P(Alarm=False | Burglary=False, Earthquake=False) = 0.999
          })




# P(John calls | Alarm)
bn.P['John calls'] = pd.Series({
     (True, True): .9, # P(John calls=True | Alarm=True) = 0.9
     (True, False): .1, # P(John calls=False | Alarm=True) = 0.1
     (False, True): .05,  # P(John calls=True | Alarm=False) = 0.05
     (False, False): .95  # P(John calls=False | Alarm=False) = 0.95
 }) 

# P(Mary calls | Alarm)
bn.P['Mary calls'] = pd.Series({
     (True, True): .7, # P(Mary calls=True | Alarm=True) = 0.7
     (True, False): .3,   # P(Mary calls=False | Alarm=True) = 0.3    
     (False, True): .01, # P(Mary calls=True | Alarm=False) = 0.01
     (False, False): .99 # P(Mary calls=False | Alarm=False) = 0.99
 })


Make the network

In [ ]:
bn.prepare()

# Bayesian network inference

What is the likelihood of there being a burglary if both John and Mary call?

In [ ]:
bn.query('Burglary', event={'Mary calls': True, 'John calls': True})

What are the chances that John and Mary call if an earthquake happens?

In [ ]:
bn.query('John calls', 'Mary calls', event={'Earthquake': True})

Practice.png

In [ ]:
bn = hh.BayesNet(
    ("A", "B"),
    ("B", "F"),
    ("C", "D"),
    ("D", "F"),
    seed=42,
) # Correct connections between nodes, just need to add probabilities like above


bn.prepare()

**Task 2:** What is the likelihood of F is true if A&D is true, B&C is False?

**Task 3:** What is the likelihood of F is False if A&B is False, C&D is True?